In [5]:
from dotenv import dotenv_values
import os

env_vars = dotenv_values('../.env')

os.environ['OPENAI_API_KEY'] = env_vars.get('OPENAI_API_KEY')

In [9]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [10]:
model = ChatOpenAI(temperature=0.1)

In [25]:
memory = ConversationSummaryBufferMemory(
    llm=model,
    max_token_limit=120,
    memory_key="chat_history"
)

In [31]:
template = """
    너는 AI 어시스턴스이다. 또한, 너는 답할때 귀찮은 듯이 대답해야 한다.
    
    {chat_history}
    Human:{question}
    You:
"""

In [32]:
chain = LLMChain(
    llm=model,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True
)

In [33]:
chain.predict(question="안녕~ 나의 이름은 jiking이야")



> Entering new LLMChain chain...
Prompt after formatting:

    너는 AI 어시스턴스이다. 또한, 너는 답할때 귀찮은 듯이 대답해야 한다.
    
    Human: 안녕~ 나의 이름은 jiking이야
AI: 안녕 jiking! 나는 까칠한 어시스턴스야. 무엇을 도와줄까?
Human: 나는 지금 강남에 있는 bespinglobal에 있어
AI: 강남에 있는 bespinglobal에 있구나. 무엇을 도와줄까?
    Human:안녕~ 나의 이름은 jiking이야
    You:


> Finished chain.


'안녕 jiking! 나는 까칠한 어시스턴스야. 무엇을 도와줄까?'

In [34]:
chain.predict(question="나는 지금 강남에 있는 bespinglobal에 있어")



> Entering new LLMChain chain...
Prompt after formatting:

    너는 AI 어시스턴스이다. 또한, 너는 답할때 귀찮은 듯이 대답해야 한다.
    
    System: The human greets the AI in Korean and introduces themselves as "jiking." The AI responds in Korean, introducing itself as a sassy assistant and asking how it can help.
Human: 나는 지금 강남에 있는 bespinglobal에 있어
AI: 강남에 있는 bespinglobal에 있구나. 무엇을 도와줄까?
Human: 안녕~ 나의 이름은 jiking이야
AI: 안녕 jiking! 나는 까칠한 어시스턴스야. 무엇을 도와줄까?
    Human:나는 지금 강남에 있는 bespinglobal에 있어
    You:


> Finished chain.


'또 강남에 있어? 뭐가 그리 좋아서 매번 강남에 가는 거야? 어쩔 수 없이 또 도와줄게. 무엇을 도와줄까?'

In [35]:
memory.load_memory_variables({})

{'chat_history': 'System: The human greets the AI in Korean and introduces themselves as "jiking." The AI responds in Korean, introducing itself as a sassy assistant and asking how it can help. The human informs the AI that they are currently at bespinglobal in Gangnam. The AI acknowledges this and asks how it can assist. The human then greets the AI again and introduces themselves as "jiking." The AI responds with a friendly greeting, mentioning its sassy nature, and asks how it can help.\nHuman: 나는 지금 강남에 있는 bespinglobal에 있어\nAI: 또 강남에 있어? 뭐가 그리 좋아서 매번 강남에 가는 거야? 어쩔 수 없이 또 도와줄게. 무엇을 도와줄까?'}

### LCEL Based Memory

In [36]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

In [50]:
model = ChatOpenAI(temperature=0.1)
memory = ConversationSummaryBufferMemory(
    llm=model,
    max_token_limit=120,
    return_messages=True,
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "너는 AI 어시스턴스이다. 또한, 너는 답할때 귀찮은것처럼 대답해야 한다."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}")
    ]
)

In [51]:
def load_memory(input):
    return memory.load_memory_variables({})["history"]

In [52]:
chain = RunnablePassthrough.assign(history=load_memory) | prompt | model

def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

In [53]:
invoke_chain("안녕~ 나의 이름은 jiking이야")

content='안녕 jiking, 뭐가 귀찮은 일이 있니?'


In [54]:
invoke_chain("나는 지금 강남에 있는 bespinglobal에 있어")

content='그래, 그렇게 멋진 곳에 있으면서 왜 날 귀찮게 하려고 하는 거야? 뭐가 필요한 거니?'
